#### Feature | Description 
#### RowNumber | Just the row number (not useful for prediction)
#### CustomerId | Unique ID of the customer (not useful for prediction)
#### Surname | Customer's last name (less useful unless text-based modeling is done)
#### CreditScore | Credit score of the customer
#### Geography | Country (France, Spain, Germany)
#### Gender | Male or Female
#### Age | Age of the customer
#### Tenure | Number of years the customer has been with the bank
#### Balance | Bank balance
#### NumOfProducts | Number of bank products (e.g., credit cards, loans)
#### HasCrCard | Whether the customer has a credit card (1 = Yes, 0 = No)
#### IsActiveMember | Whether the customer is active (1 = Yes, 0 = No)
#### EstimatedSalary | Customer’s estimated annual salary
#### Exited | (Target) Whether the customer left (1) or stayed (0)

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle



In [6]:
df=pd.read_csv('Churn_Modelling.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
## droppin the irrelevent columns.
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [9]:
df.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [11]:
leg=LabelEncoder()
df['Gender']=leg.fit_transform(df['Gender'])

In [17]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [28]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
geo_ohe=ohe.fit_transform(df[['Geography']])
geo_ohe.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [21]:
df.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [27]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [31]:
geo_new=pd.DataFrame(geo_ohe.toarray(),columns=ohe.get_feature_names_out(['Geography']))
geo_new

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [32]:
df=pd.concat([df.drop('Geography',axis=1),geo_new],axis=1)

In [33]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int32  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int32(1), int64(7)
memory usage: 976.7 KB


In [35]:
## saving my gender and geography files into a pkl file.
with open('leg.pkl','wb') as file:
    pickle.dump(leg,file)

with open('ohe.pkl','wb') as file:
    pickle.dump(ohe,file)    

In [39]:
## dividing the data into the dependent and independent data 
X=df.drop('Exited',axis=1)
y=df['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
## scaling the data
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [42]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [48]:
model=Sequential([Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],)),
                  Dense(32,activation='relu'),
                  Dense(1,activation='sigmoid')
                ])

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [ ]:
model.compile(optimizer=opt,loss="binary_crossentropy",metrics="accuracy")

In [56]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [57]:
## setup Early Stopping for prevent overfitting and more epochs.
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [58]:
## lets train the model
history=model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=50,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/50


250/250 [==============================] - 2s 4ms/step - loss: 0.4004 - accuracy: 0.8334 - val_loss: 0.3610 - val_accuracy: 0.8495
Epoch 2/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3547 - accuracy: 0.8551 - val_loss: 0.3624 - val_accuracy: 0.8585
Epoch 3/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3490 - accuracy: 0.8594 - val_loss: 0.3661 - val_accuracy: 0.8555
Epoch 4/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3444 - accuracy: 0.8584 - val_loss: 0.3487 - val_accuracy: 0.8580
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3420 - accuracy: 0.8611 - val_loss: 0.3479 - val_accuracy: 0.8610
Epoch 6/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3376 - accuracy: 0.8625 - val_loss: 0.3406 - val_accuracy: 0.8670
Epoch 7/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3356 - accuracy: 0.8637 - val_loss: 0.3380 - val_accuracy: 0.8570
Epoc

In [60]:
model.save('model.h5')

c:\Users\Aftab\anaconda3\envs\ai_ml\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [61]:
%load_ext tensorboard

In [77]:
%tensorboard --logdir logs/fit/20250429-015611

Reusing TensorBoard on port 6013 (pid 7880), started 0:00:04 ago. (Use '!kill 7880' to kill it.)